In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Prepare dataset
data = np.loadtxt('data.csv', delimiter=',', dtype=float)
labels = data[:, 0]
features = preprocessing.minmax_scale(data[:, 1:])
X_train, X_test, y_train, y_test = train_test_split(features, labels.ravel(), test_size=0.3)

### 의사결정나무
* random_state = 2022 으로 설정
* 변수명은 dt_clf 로 설정

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
# 빈 부분 코드 작성
# decision tree
dt_clf = DecisionTreeClassifier(random_state=2022)

In [5]:
# 개별 분류기에 train set 피팅
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=2022)

In [6]:
# test셋으로 prediction
dt_pred = dt_clf.predict(X_test)

In [7]:
# 성능 확인
# accuracy_score 계산
print(f'{accuracy_score(y_test, dt_pred): .4f}')

accuracy_score(y_test, dt_pred): .4f


### 랜덤 포레스트
* random_state = 2022
* 변수명 rf_clf 로

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
# random forest
rf_clf = RandomForestClassifier(random_state=2022)

In [10]:
# 개별 분류기에 train set 피팅
rf_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=2022)

In [11]:
# test셋으로 prediction
rf_pred = rf_clf.predict(X_test)

In [12]:
# 성능 확인
# accuracy_score 계산
print(f'{accuracy_score(y_test, rf_pred): .4f}')

accuracy_score(y_test, rf_pred): .4f


### Gradient Boost
* random_state = 2022
* 변수명 gb_clf 로

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

In [14]:
# gradient boost
gb_clf = GradientBoostingClassifier(random_state=2022)

In [15]:
# 개별 분류기에 train set 피팅
gb_clf.fit(X_train, y_train)

GradientBoostingClassifier(random_state=2022)

In [16]:
# test셋으로 prediction
gb_pred = gb_clf.predict(X_test)

In [17]:
# 성능 확인
# accuracy_score 계산
print(f'{accuracy_score(y_test, gb_pred): .4f}')

accuracy_score(y_test, gb_pred): .4f


### 보고서 과제
1. voting, bagging, random forest, boosting, adaboost, gradient boost 의 특징 및 장단점을 스스로 정리해보기
2. Boosting의 advanced model 인 XGBoost, LightGBM, CatBoost에 대해 찾아보고 정리해보기

#### 기본 Ensemble 모델

|기준|Voting|Bagging|Random Forest|
|:---:|:---:|:---:|:---:|
|Dataset 구성|그대로 사용|Bootstrap|Bootstrap|
|Instance 모델 구성|서로 다른 모델 구성|Decision Tree<br>(**동일 변수**)|Decision Tree<br>(**변수 추출**)|
|장점||모델의 variance를 줄여서 overfitting 방지|한 모델 내 변수 간 상관성이 낮아져 **모델 성능의 변동이 감소** <br> 변수 중요도를 통해 어떤 독립 변수가 중요한지 정량적인 비교가능|
|특징|Hard(Mode)<br> **Soft(argmax(Avg))**| OOB<br>(남은 데이터는 test 데이터로) | OOB<br>(남은 데이터는 test 데이터로)|

세 앙상블 모델에서 공통점은 다음과 같습니다.

데이터 셋을 달리하든, 변수를 달리하든, 애초에 모델을 달리하여서 여러 모델을 학습시키고,
하나의 결론을 짓는 것이 앙상블 모델의 기본 원리입니다. 

그렇기 때문에 여러 모델의 결과를 하나로 합치는 방법이 모두 필요합니다. 
범주형 데이터의 경우 mode를 사용하기도 하고 수치형 데이터의 경우에는 평균을 사용할 수 있습니다.

하지만 이런 방식의 경우 최종 결과에 대한 해석력에 한계가 있다는 단점이 존재하게 됩니다. 

|기준|Boosting|Adaboost|Gradient Boost|
|:---:|:---:|:---:|:---:|
|Dataset 구성||오분류 데이터에 높은 weight를 가지는 **weighted sampling**으로 구성|이전 학습의 **residual**을 타겟으로 학습|
|Instance 모델 구성 |동일한 분류기|약 분류기(Weak Classifier)|약분류기|
|특징||최종 결과는 weighted sum으로 aggregation|성능은 좋아질 수 있으나 오버피팅에 취약|

세 부스팅 모델에서 공통점은 다음과 같습니다.

순차적으로 학습을 진행하여 오분류한 데이터에 대해 가중치를 주어 학습을 진행하는 방식입니다.

다만 순차적으로 학습을 진행하기 때문에 **학습시간이 길다**는 단점이 존재하긴 합니다.

#### XGBoost, LightGBM, CatBoost 요약

> XGBoost

앞서 Boosting 알고리즘의 한계였던 순차적인 학습 방법으로 인해 병렬 처리가 불가능한 부분을 극복했다는 점이 큰 특징입니다. 

병렬처리를 진행하는 방식은 다음과 같습니다.
1. 기존의 node에서 feature split을 찾는 과정을 병렬처리
2. 기존의 node에서 어떤 feature를 split할 것인지 순서를 매기는 것을 병렬처리

이 외에도 Gradient Boosting Model(GBM)과 차이점을 몇가지 정리하면 다음과 같습니다.
 - (Regularization) L1이나 L2 norm을 추가하여 overfitting을 방지합니다.
 - (Tree Pruning) 미리 셋팅해둔 max_depth까지만 split하고 pruning을 하며거꾸로 올라가면서 positive gain이 없는 노드를 삭제합니다.
 - Cross validation이 내장되어 있습니다.

> LightGBM

이전의 XGBoost까지는 같은 depth에서 split을 진행하고 다음 level로 넘어가는 방식(**level-wise**)이었다면, LigthGBM의 경우엔 node split이 leaf마다 진행(**leaf-wise**)됩니다. 이렇게 할 경우 모든 leaf를 다 나누지 않게 되어 시간이나 메모리에서 개선이 있었습니다.

이 외에도 특징은 다음과 같습니다.
- gradient가 클수록 덜 학습이 된 것이니 information gain이 많다고 생각하여 gradient가 적은 데이터 일부분을 사용하지 않는 방식의 **GOSS(Graident-based One Side Sampling)** 을 사용합니다.

- feature들 중에서 서로 상호 베타적인 feature의 경우 하나의 feature로 사용하여 훈련속도를 향상시키는 **EFB(Exclusive Feature Bundling)** 을 사용합니다.

다만 데이터 셋이 작은 경우 과적합에 취약하다는 단점이 있습니다.

> CatBoost(Categorical Boosting)

기존 알고리즘과 달리 이름 그대로 범주형 데이터를 처리할 수 있는 모델입니다.

변수형 데이터를 처리하기 위해 One-hot encoding처럼 Target Encoding, Mean Encoding으로 불리는 방식을 사용합니다. 이러한 방식은 target에 대한 정보가 사용되는 data leakage문제가 있어 순차적으로 데이터를 학습할 때 이전까지 사용된 데이터의 타겟 정보만 사용하여 변수형 데이터를 처리합니다. 그리고 범주형 데이터를 몇가지로 묶어서 다시 사용하여 EFB와 비슷한 방식도 사용합니다.

또한 과적합한다는 한계점을 해결하기 위해 **ordered Boosting**이란 방식을 사용합니다.

Ordered Boosting이란 기존엔 전체 feature를 사용하고 이에 대한 error를 순차적으로 학습을 진행한다면 본 방식은 사용하는 feature도 순차적으로 늘려나가는 방식으로 진행된다는 것입니다. 또한 이렇게 feature를 순차적으로 사용할 때의 순서 또한 random하게 여러번 뽑아서 사용하여 순서에 의한 randomness를 해소합니다.

다만 데이터 셋에 수치형 데이터가 많다면 다른 모델보다 학습속도가 느리다는 단점이 존재합니다. 

---

특징을 정리하면 다음과 같습니다.

|특징|XGBoost|LightGBM|CatBoost|
|:---:|:---:|:---:|:---:|
|Dataset 구성||GOSS<br>EFB|데이터를 순차적으로 사용한 범주형 데이터 인코딩<br> 중복되는 범주형 데이터 동일하게 처리|
|트리 생성 방식|Level-wise|Leaf-wise|Level-wise|
|과적합 예방|tree pruning||Ordered Boosting|